<a href="https://colab.research.google.com/github/Timurpy/senescence/blob/master/Gene_of_senescence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pylab as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats
import datetime as dt
import os
import random
from IPython.display import display

sns.set_palette(sns.color_palette('deep'))
sns.set_style('whitegrid')

plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 14

pd.set_option('display.max_colwidth', 80)

In [26]:
cellage_df = pd.read_csv('cellAge1.csv', sep=';')

In [27]:
cellage_df.head()

,hagrid,gene_name,hgncid,entrezid,organism,cancer_type,senescence_effect,description,notes
0,1,ACLY,115,47,Human,No,Inhibits,ACLY knockdown in primary human cells triggered cellular senescence and acti...,"The primary HDF cells, HEK293T and HCT116 cells"
1,2,AAK1,19679,22848,Human,No,Induces,A library of 200 constitutively activated kinases was screened in order to i...,"IMR-90, MRC-5, and GP293 cells"
2,3,ABI3,29859,51225,Human,Yes,Induces,"Ectopic expression of ABI3 lowered transforming activity, reduced in vitro t...",A follicular thyroid carcinoma cell line (WRO) and a colon cancer-derived HT...
3,4,ADCK5,21738,203054,Human,No,Induces,A library of 200 constitutively activated kinases was screened in order to i...,"IMR-90, MRC-5 , and GP293 cells"
4,5,AKR1B1,381,231,Human,Yes,Inhibits,Overexpression of aldose reductase (AKR1B1) in HaCaT cells attenuated UVB-in...,HaCaT cells


In [9]:
genes_df = pd.read_csv('https://raw.githubusercontent.com/tomquisel/senescence/master/data/gene_info.txt', sep='\t', dtype={'entrez_id': 'int'})

In [10]:
genes_df.columns = ['hgnc_id', 'symbol', 'name', 'status', 'prev_symbols', 'synonyms', 'chromosome', 'accession_numbers', 'refseq_ids', 'entrez_id']

In [11]:
genes_df = genes_df.query('entrez_id == entrez_id')

In [12]:
genes_df['entrez_id'] = genes_df.entrez_id.astype('int')

In [13]:
genes_df.head()

,hgnc_id,symbol,name,status,prev_symbols,synonyms,chromosome,accession_numbers,refseq_ids,entrez_id
0,HGNC:5,A1BG,alpha-1-B glycoprotein,Approved,NaN,NaN,19q13.43,NaN,NM_130786,1
1,HGNC:37133,A1BG-AS1,A1BG antisense RNA 1,Approved,"NCRNA00181, A1BGAS, A1BG-AS",FLJ23569,19q13.43,BC040926,NR_015380,503538
2,HGNC:24086,A1CF,APOBEC1 complementation factor,Approved,NaN,"ACF, ASP, ACF64, ACF65, APOBEC1CF",10q11.23,AF271790,NM_014576,29974
3,HGNC:7,A2M,alpha-2-macroglobulin,Approved,NaN,"FWP007, S863-7, CPAMD5",12p13.31,"BX647329, X68728, M11313",NM_000014,2
4,HGNC:27057,A2M-AS1,A2M antisense RNA 1 (head to head),Approved,NaN,NaN,12p13.31,NaN,NR_026971,144571


In [16]:
def entrez_ids_to_symbols(ids):
    df = genes_df.set_index('entrez_id')
    return df.loc[ids].symbol.values

In [17]:
entrez_ids_to_symbols([1, 2])

array(['A1BG', 'A2M'], dtype=object)

In [28]:
cellage_df.senescence_effect.value_counts()

Induces     153
Inhibits    121
Unclear       5
Name: senescence_effect, dtype: int64

In [32]:
cellage_df.organism.value_counts()

Human    279
Name: organism, dtype: int64

In [33]:
genes_df.status.value_counts()

Approved    41095
Name: status, dtype: int64

In [34]:
%%bash
wget http://geneontology.org/ontology/go-basic.obo

--2023-03-08 19:33:53--  http://geneontology.org/ontology/go-basic.obo
Resolving geneontology.org (geneontology.org)... 34.233.67.155
Connecting to geneontology.org (geneontology.org)|34.233.67.155|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://purl.obolibrary.org/obo/go/go-basic.obo [following]
--2023-03-08 19:33:53--  http://purl.obolibrary.org/obo/go/go-basic.obo
Resolving purl.obolibrary.org (purl.obolibrary.org)... 3.223.180.112
Connecting to purl.obolibrary.org (purl.obolibrary.org)|3.223.180.112|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://current.geneontology.org/ontology/go-basic.obo [following]
--2023-03-08 19:33:53--  http://current.geneontology.org/ontology/go-basic.obo
Resolving current.geneontology.org (current.geneontology.org)... 108.156.61.208, 108.156.61.101, 108.156.61.32, ...
Connecting to current.geneontology.org (current.geneontology.org)|108.156.61.208|:80... connected.
HTTP request sen

In [35]:
obo_df = pd.read_csv('https://raw.githubusercontent.com/tomquisel/senescence/master/data/go-basic.obo', sep='\n')

In [36]:
obo_df.head()

,format-version: 1.2
0,data-version: releases/2017-09-20
1,"subsetdef: goantislim_grouping ""Grouping classes that can be excluded"""
2,"subsetdef: gocheck_do_not_annotate ""Term not to be used for direct annotation"""
3,"subsetdef: gocheck_do_not_manually_annotate ""Term not to be used for direct ..."
4,"subsetdef: goslim_agr ""AGR slim"""


In [37]:
%%bash
wget -nc  ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz

--2023-03-08 19:35:23--  ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz
           => ‘gene2go.gz’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 165.112.9.228, 2607:f220:41f:250::230, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /gene/DATA ... done.
==> SIZE gene2go.gz ... 27310876
==> PASV ... done.    ==> RETR gene2go.gz ... done.
Length: 27310876 (26M) (unauthoritative)

     0K .......... .......... .......... .......... ..........  0% 94.6K 4m41s
    50K .......... .......... .......... .......... ..........  0%  553K 2m45s
   100K .......... .......... .......... .......... ..........  0% 57.1M 1m50s
   150K .......... .......... .......... .......... ..........  0%  557K 94s
   200K .......... .......... .......... .......... ..........  0% 63.1M 75s
   250K .......... .......... .......... ..

In [42]:
%%bash
echo "# Gene_of_senescence" >> README.md
git init
git add README.md
git commit -m "first commit"
git branch -M main
git remote add origin https://github.com/Timurpy/Gene_of_senescence.git
git push -u origin main

Initialized empty Git repository in /content/.git/



*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@e1b801c652ee.(none)')
error: refname refs/heads/master not found
fatal: Branch rename failed
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/Timurpy/Gene_of_senescence.git'


CalledProcessError: ignored

In [44]:
git config --global user.email "timerlan1963@gmail.com"
git config --global user.name "Timurpy"

SyntaxError: ignored

In [40]:
from goatools.obo_parser import GODag

obodag = GODag("/content/go-basic.obo")

ModuleNotFoundError: ignored

In [38]:
from goatools.base import gobasic.obo
obo_fname = gobasic.obo()

SyntaxError: ignored

In [ ]:
from goatools.obo_parser import GODag

obodag = GODag("gobasic.obo")

ModuleNotFoundError: ignored

In [ ]:
from goatools.associations import read_ncbi_gene2go

ModuleNotFoundError: ignored

In [ ]:
from goatools.obo_parser import GODag
from goatools.base import download_ncbi_associations
gene2go = download_ncbi_associations()

ModuleNotFoundError: ignored

In [ ]:
from goatools.obo_parser import GODag

obodag = GODag("data/go-basic.obo")

In [ ]:
from goatools.associations import read_ncbi_gene2go

ModuleNotFoundError: ignored

In [ ]:
Get http://geneontology.org/ontology/go-basic.obo
from goatools.base import download_go_basic_obo
obo_fname = download_go_basic_obo()

SyntaxError: ignored

In [ ]:
from goatools.obo_parser import GODag

obodag = GODag("")

ModuleNotFoundError: ignored

In [ ]:
from goatools.obo_parser import GODag

obodag = GODag("from goatools.obo_parser import GODag")


ModuleNotFoundError: ignored